### STEP.1-匯入檔案

In [40]:
import requests
url="https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON"

respone=requests.get(url)

### STEP.2-讀取檔案

In [41]:
data=respone.json()

### STEP.3-使用迴圈依序將值存入串列中

In [42]:
eachdata=[]
#items為字典
for items in data["records"]:
    #取出所需的欄位值，並將這些值組成一個元組，再將該元組添加到串列中
    eachdata.append((items['sitename'],
                        items['county'],
                        items['aqi'],
                        items['status'],
                        items['pm2.5'],
                        items['datacreationdate'],
                        items['latitude'],
                        items['longitude']))
    
    # print(items)
print(eachdata)


[('屏東(枋山)', '屏東縣', '74', '普通', '10', '2024-11-02 16:00', '22.260899', '120.651472'), ('宜蘭（頭城）', '宜蘭縣', '35', '良好', '1', '2024-11-02 16:00', '24.85366672', '121.82463916'), ('新北(樹林)', '新北市', '25', '良好', '7', '2024-11-02 16:00', '24.94902778', '121.38352778'), ('屏東（琉球）', '屏東縣', '82', '普通', '18', '2024-11-02 16:00', '22.35222', '120.37722'), ('臺南（麻豆）', '臺南市', '43', '良好', '14', '2024-11-02 16:00', '23.17904722', '120.24583056'), ('高雄（湖內）', '高雄市', '42', '良好', '4', '2024-11-02 16:00', '22.87985556', '120.24535'), ('彰化（員林）', '彰化縣', '41', '良好', '9', '2024-11-02 16:00', '23.96117', '120.56373'), ('大城', '彰化縣', '47', '良好', '13', '2024-11-02 16:00', '23.85493056', '120.26964167'), ('富貴角', '新北市', '46', '良好', '9', '2024-11-02 16:00', '25.29681695', '121.53656894'), ('麥寮', '雲林縣', '45', '良好', '12', '2024-11-02 16:00', '23.753506', '120.251825'), ('關山', '臺東縣', '28', '良好', '5', '2024-11-02 16:00', '23.045083', '121.161933'), ('馬公', '澎湖縣', '44', '良好', '9', '2024-11-02 16:00', '23.569031', '119.566158'), 

### STEP.4-新增資料庫

In [43]:
sql = '''
CREATE TABLE IF NOT EXISTS records (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	sitename TEXT NOT NULL,
	county TEXT,
	aqi INTEGER,
	status TEXT,
	pm25 NUMERIC,
	date TEXT,
	lat NUMERIC,
	lon NUMERIC,
    UNIQUE (sitename,county)
);
'''

import sqlite3
conn = sqlite3.connect("AQI_issue.db")

cursor = conn.cursor()
cursor.execute(sql)
conn.commit()
cursor.close()
conn.close()

### STEP.5-匯入資料

In [45]:
data = ('Site Name', 'County Name', 60, '普通', 17, "2024-11-02 15:00"
, 22.260899
, 120.651472)

insertSQL = '''
INSERT OR REPLACE INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
VALUES (?,?,?,?,?,?,?,?)
'''
conn = sqlite3.connect("AQI_issue.db")

cursor = conn.cursor()
#使用executemany插入多筆資料
cursor.executemany(insertSQL,eachdata)
conn.commit()
cursor.close()
conn.close()